In [1]:
import numpy as np
import pandas as pd
import math 
import regex as re
from datetime import datetime
import timedelta as td
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
def dirImagens(pDirImagem):
    try:
        if (not os.path.isdir(pDirImagem)):
            os.mkdir(pDirImagem)
    except OSError as e:
        print(e.args[1])
    
def abriPlanilha():
    diretorio='./data/'
    for arquivo in os.listdir(diretorio):
        if (re.findall('csv', arquivo)):
            return (diretorio+arquivo)
    
dirImagens('Imagens')
    
ins = pd.read_csv(abriPlanilha()
                 ,header=1
                 ,sep=','
                 ,low_memory=True
                 )

In [3]:
colunas=['col1','col2','col3','situacao','nroInscricao','nome',
        'cpf','dataNascimento','genero','escolaridade',
        'instituicao','telefone','email','cep','bairro',
        'cidade','incluiOutro','perfil','observacao','perfis']

ins.columns=colunas

In [4]:
def columnsDrop():
    ins.drop('situacao', axis=1, inplace=True)
    ins.dropna(axis=1, how='all', inplace=True)
    ins.dropna(axis=0, how='all', inplace=True)

columnsDrop()

In [5]:
def columnsInsert():
    ins.insert(3, 'cpfAjustado', '')
    ins.insert(5, 'idade', 0)
    ins.insert(6, 'dataNascAjustada', ' ')
    ins.insert(8, 'generoId', 99)
    ins.insert(10, 'escolaridadeId', 0)
    ins.insert(11, 'escolaridade2', ' ')
    ins.insert(13, 'instituicaoId', 0)
    ins.insert(15, 'telefoneAjustado', '')
    ins.insert(17, 'emailValidado', '')
    ins.insert(19, 'cepAjustado', '')
    ins.insert(22, 'cidadeId', 0)
    ins.insert(26, 'tamanhoCamisetaId', 0)

columnsInsert()

In [6]:
ins.info()

<class 'pandas.core.frame.DataFrame'>
Index: 58 entries, 0 to 57
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   nroInscricao       58 non-null     object
 1   nome               58 non-null     object
 2   cpf                57 non-null     object
 3   cpfAjustado        58 non-null     object
 4   dataNascimento     57 non-null     object
 5   idade              58 non-null     int64 
 6   dataNascAjustada   58 non-null     object
 7   genero             58 non-null     object
 8   generoId           58 non-null     int64 
 9   escolaridade       58 non-null     object
 10  escolaridadeId     58 non-null     int64 
 11  escolaridade2      58 non-null     object
 12  instituicao        50 non-null     object
 13  instituicaoId      58 non-null     int64 
 14  telefone           57 non-null     object
 15  telefoneAjustado   58 non-null     object
 16  email              58 non-null     object
 17  emai

In [7]:
ins.head(3)

,nroInscricao,nome,cpf,cpfAjustado,dataNascimento,idade,dataNascAjustada,genero,generoId,escolaridade,...,emailValidado,cep,cepAjustado,bairro,cidade,cidadeId,incluiOutro,perfil,observacao,tamanhoCamisetaId
0,1,Isabelle Victoria Pereira De Souza,51454217880,,12/09/2003,0,,Feminino,99,Ensino Superior,...,,08121-620,,Jd Nélia,São Paulo,0,N,Desenvolvedor,Tamanho de Camiseta: M,0
1,2,Guilherme Antunes de Campos,50069148821,,03/06/2004,0,,Masculino,99,Ensino Superior Cursando,...,,7091000,,Vila Progresso,Guarulhos,0,N,Desenvolvedor,Tamanho de Camiseta: G,0
2,3,Wezilyana Melice Farias de Lima,1264343442,,13/08/1983,0,,Feminino,99,Pós-graduação completo,...,,1529001,,Aclimação,São Paulo,0,N,Analista de Negócios,Tamanho de Camiseta: M,0


In [8]:
def cpfAjustado(cpf) -> str:
    pattern='([0-9]{3})([.])([0-9]{3})([.])([0-9]{3})([-])([0-9]{2})'
    if (re.search(pattern, cpf)):
        return cpf
    
    cpf=re.sub('[\.\-]','',cpf); cpf=cpf.zfill(11)
        
    pattern='([0-9]{3})([0-9]{3})([0-9]{3})([0-9]{2})'
    cpf=re.search(pattern, cpf)
    cpf='{}.{}.{}-{}'.format(cpf.group(1)
                            ,cpf.group(2)
                            ,cpf.group(3)
                            ,cpf.group(4)
                            )
    return cpf

def dataNascAjustada(pDataNascimento : str, éIdade : bool = False):
    dtNasc=pDataNascimento; dtRef=''
    #   
    pattern='([0-9]{1,2})([/])([0-9]{1,2})([/])([0-9]{2,4})'
        
    if (not re.search(pattern, dtNasc)):
        if (len(dtNasc)==6 or len(dtNasc)==8):
            pattern='([0-9]{2})([0-9]{2})([0-9]{2,4})'
            dtNasc=re.search(pattern, dtNasc)
            if (dtNasc):
                dtNasc='{}/{}/{}'.format(dtNasc.group(1),
                                        dtNasc.group(2),
                                        dtNasc.group(3)
                                        )
            else:
                return -1
            
        else:
            return -1
        
    dtNasc=datetime.strptime(dtNasc, '%d/%m/%Y')
                
    if (éIdade):
        diferencaData=(datetime.today()-dtNasc)
        dias=diferencaData.days
        anos, dias = divmod(dias, 365)
        dtRef=anos
    else:
        dtRef=dtNasc.strftime('%d/%m/%Y %H:%M:%S')
    
    return dtRef

def quantGenero(genero) -> int:
    if (genero.upper()=='FEMININO'):
        return 1
    elif (genero.upper()=='MASCULINO'):
        return 2
    else:
        return 99
    
def quantEscolaridade(escolaridade) -> int:
    escolaridade=escolaridade.upper()
    valor=0
    if re.findall('M[EÉ]DIO', escolaridade):
        valor=10
    elif (re.findall('SUPERIOR', escolaridade) or
         re.findall('FACULDADE', escolaridade)):
        valor=20
    elif re.findall('P[ÓO]S', escolaridade):
        valor=30
    elif re.findall('MESTRADO', escolaridade):
        valor=40
    elif re.findall('DOUTORADO', escolaridade):
        valor=50
    else:
        return valor
          
    if (not (re.findall('INCOMPLETO', escolaridade) or 
        re.findall('CURSANDO', escolaridade))):
        valor+=5
        
    return valor

def qualEscolaridade(escolaridade) -> str:
    escolaridade=escolaridade.upper()
    texto='N/D'
    if re.findall('M[EÉ]DIO', escolaridade):
        texto='ENS.MÉDIO'
    elif (re.findall('SUPERIOR', escolaridade) or
         re.findall('FACULDADE', escolaridade) or
         re.findall('BACHAREL', escolaridade)
         ):
        texto='ENS.SUPERIOR'
    elif re.findall('P[ÓO]S', escolaridade):
        texto='PÓS-GRADUAÇÃO'
    elif re.findall('MESTRADO', escolaridade):
        texto='MESTRADO'
    elif re.findall('DOUTORADO', escolaridade):
        texto='DOUTORADO'
    else:
        return texto
          
    if (not (re.findall('INCOMPLETO', escolaridade) or 
        re.findall('CURSANDO', escolaridade) or
        re.findall('ANDAMENTO', escolaridade))):
        texto=texto+' COMPLETO'
    else:
        texto=texto+' INCOMPLETO/CURSANDO'
        
    return texto

def telefoneAjustado(fone) -> str:
    #print('Telefone: {}'.format(fone))

    if (pd.isnull(fone)):
        #print('Não informado!'); print('----------------')
        return 'Não informado!'
    
    pattern='([\(])([0-9]{2})([\)])([\ ])([0-9]{5})([-])([0-9]{4})'
    if (re.search(pattern, fone)):
        #print('foneCond.1: '+fone); print('----------------')
        return fone

    for char in ([' ','-','\(','\)']):
        fone=re.sub(char,'',fone)
    
    pattern='([0-9]{2})([0-9]{5})([0-9]{4})'
    if (re.search(pattern, fone) and (not re.search('^55', fone))): 
        fone=re.search(pattern, fone)
        fone='({}) {}-{}'.format(fone.group(1)
                                 ,fone.group(2)
                                 ,fone.group(3)
                                )
        #print('foneCond.2: '+fone); print('----------------')
    elif (re.search('^55',fone)):
        pattern = '([0-9]{2})([0-9]{2})([0-9]{5})([0-9]{4})'
        fone=re.search(pattern, fone)
        fone='({}) {}-{}'.format(
                                fone.group(2)
                                ,fone.group(3)
                                ,fone.group(4)
                                )
        #print('foneCond.3: '+fone); print('----------------')
    else:
        #print('Inválido'); print('----------------')
        fone='Inválido'

    return fone

def validarEmail(email):
    # https://www.geeksforgeeks.org/check-if-email-address-valid-or-not-in-python/
    pattern='(^[a-zA-Z0-9_.-]{4,40})(@){1}([a-zA-Z0-9-]+)(\.){1}([a-zA-Z0-9-.]+$)'
    email=re.search(pattern, email)
    #print(email)
    if (not email):
        return [False, 'Formação do E-Mail incorreta!']
    #print(email.group(2), email.start(), email.end()) ## @
    return [True, str.lower(email.group(0))]

def quantInstituicao(pInstituicao) -> int:
    tp=type(pInstituicao)
    if (tp==str):
        pInst=pInstituicao.upper()
        
        if (re.findall('BELAS ARTES', pInst)):
            return 1
        elif (re.findall('FATEC', pInst)):
            return 2
        elif (re.findall('WYDEN', pInst)):
            return 3
        elif (re.findall('JUDAS', pInst)):
            return 4
        elif (re.findall('IFSP', pInst) or re.findall('INSTITUTO FEDERAL', pInst)):
            return 5
        elif (re.findall('ENIAC', pInst)):
            return 6
        elif (re.findall('FIAP', pInst)):
            return 7
        elif (re.findall('FAM', pInst)):
            return 8
        elif (re.findall('UNICAMP', pInst)):
            return 9
        elif (re.findall('ETEC', pInst)):
            return 10
        elif (re.findall('LICEU', pInst)):
            return 11
        elif (re.findall('MOACYR CAMPOS', pInst)):
            return 12
        elif (re.findall('USP', pInst) or re.findall('UNIVERSIDADE DE S[AÃ]O PAULO', pInst)):
            return 13
        elif (re.findall('DRUMMOND', pInst)):
            return 14
        elif (re.findall('IMPACTA', pInst)):
            return 15
        elif (re.findall('VUNESP', pInst)):
            return 16
        elif (re.findall('SAL[EÉ]M', pInst)):
            return 17
        elif (re.findall('TORRICELLI', pInst)):
            return 18
        else:
            return 99
      
    elif (tp==float):
        return 0

    return 99

def cepAjustado(cep) -> str:
    pattern='([0-9]{5})([-]{1})([0-9]{3})'
    if (not re.search(pattern, cep)):
        cep=re.sub('-','', cep)
        cep=cep.zfill(8)
        pattern='([0-9]{5})([0-9]{3})'
        cep=re.search(pattern, cep)
        cep='{}-{}'.format(cep.group(1),cep.group(2))
    return cep

def cidadeMaiuscula(cidade) -> str:
    return str.upper(cidade)

def quantCidade(cidade) -> int:
    cidade=cidade.upper()
    if re.findall('GUARULHOS', cidade):
        return 1
    elif re.findall('S[ÃA]O PAULO', cidade):
        return 2
    elif re.findall('RIBEIR[ÃA]O PRETO', cidade):
        return 3
    elif re.findall('SANTO ANDR[ÉE]', cidade):
        return 4
    elif re.findall('GUARUJ[ÁA]', cidade):
        return 5
    elif re.findall('IGARAT[ÁA]', cidade):
        return 6
    elif re.findall('CARAPICU[ÍI]BA', cidade):
        return 7
    elif re.findall('S[ÃA]O VICENTE', cidade):
        return 8
    else:
        return 99

def quantTamanhoCamiseta(pTamanho) -> int:
    t=re.split('\s',pTamanho)
    if (t[len(t)-1]=='P'):
        return 1
    elif (t[len(t)-1]=='M'):
        return 2
    elif (t[len(t)-1]=='G'):
        return 3
    elif (t[len(t)-1]=='GG'):
        return 4
    else:
        return 99    


In [9]:
#ins['cpfAjustado']=ins['cpf'].apply(lambda cpf: cpfAjustado(cpf))
ins['idade']=ins['dataNascimento'].apply(lambda dtNasc: dataNascAjustada(dtNasc, True))
ins['dataNascAjustada']=ins['dataNascimento'].apply(lambda dtNasc: dataNascAjustada(dtNasc, False))
ins['generoId']=ins['genero'].apply(lambda genero: quantGenero(genero))
ins['escolaridadeId']=ins['escolaridade'].apply(lambda escolaridade: quantEscolaridade(escolaridade))
ins['escolaridade2']=ins['escolaridade'].apply(lambda escolaridade: qualEscolaridade(escolaridade))
ins['instituicaoId']=ins['instituicao'].apply(lambda instituicao: quantInstituicao(instituicao))
ins['telefoneAjustado']=ins['telefone'].apply(lambda fone: telefoneAjustado(fone))
ins['emailValidado']=ins['email'].apply(lambda email : validarEmail(email)[1])
ins['cepAjustado']=ins['cep'].apply(lambda cep : cepAjustado(cep))
ins['cidade']=ins['cidade'].apply(lambda cidade : cidadeMaiuscula(cidade))
ins['cidadeId']=ins['cidade'].apply(lambda cidade: quantCidade(cidade))
ins['tamanhoCamisetaId']=ins['observacao'].apply(lambda obs: quantTamanhoCamiseta(obs))

TypeError: expected string or buffer

In [ ]:
ins

In [ ]:
X = ins['escolaridade2']
y = ins['idade']

### Gráficos

In [ ]:
def grafico1(X, y):
    plt.figure(figsize=(10,8))
    plt.title('Escolaridade por Idade', fontdict={'weight':'bold'})
    plt.scatter(X,y,c='darkblue',marker='o')
    plt.xlabel('Escolaridade', fontdict={'weight':'bold','size':16})
    plt.ylabel('Idade', fontdict={'weight':'bold','size':16})
    plt.xticks(rotation=60)
    plt.show()
    
    
def grafico2(X, y, nomeArquivo):
    plt.figure(figsize=(12,8))
    plt.title('Escolaridade por Idade', fontdict={'weight':'bold'})
    plt.bar(X, y, width=1, color='darkblue', edgecolor="white", linewidth=1)
    plt.xlabel('Escolaridade', fontdict={'weight':'bold','size':16})
    plt.ylabel('Idade', fontdict={'weight':'bold','size':16})
    plt.xticks(rotation=60)
    plt.savefig(nomeArquivo, dpi=300)
    plt.show()
    
grafico2(X, y, './Imagens/fig1.jpg')

In [ ]:
# Monta um gráfico de Pizza, com contagem de valores.
def dataPieGraph(dfColuna) -> list: 
    etiquetas=[]; valores=[]
    
    for descricao in dfColuna:
        p=0; localizado=False
        for et in etiquetas:
            if (descricao==et):
                valores[p]+=1
                localizado=True
                break
            else:
                p+=1
        if (not localizado):
            etiquetas.append(descricao)
            valores.append(1)
    
    return [etiquetas, valores]

def dataComputeByRange(faixa, valores) -> list:
    # Array de retorno.
    ret=[]; etiquetas=[]
    for fx in range(len(faixa)):
        ret.append(0)
    #print(ret)

    for etq in faixa:
        if (len(etq)>1):
            etiquetas.append('De {min} à {max}.'.format(min=etq['min'], max=etq['max']))
        else:
            etiquetas.append('{acima}+'.format(acima=etq['acima']))

    for valor in valores:
        for posicao, fx in enumerate(faixa):
            if (len(fx)==1):
                acimaDe=fx['acima']
                if (valor >= acimaDe):
                    ret[posicao]+=1
                    #print('Posição: {}, Valor: {}, Faixa: {}'.format(posicao, valor, fx))
            else:
                min=fx['min']; max=fx['max']
                if (valor >= min and valor <= max):
                    ret[posicao]+=1
                    #print('Posição: {}, Valor: {}, Faixa: {}'.format(posicao, valor, fx))
    return [etiquetas, ret]
    
def pieGraph(dataPie, **kargs) -> None:
    etiquetas=dataPie[0]
    valores=dataPie[1]
    
    # **kargs
    titulo='Titulo'; nomeArquivo='./Imagem'+str.zfill(str(np.random.randint(999)),3)
    colormap='viridis'
        
    for chave in kargs:
        if (str.lower(chave)=='titulo'):
            titulo=kargs['titulo']
        if (str.lower(chave)=='nomearquivo'):
            nomeArquivo=kargs[chave]
        if (str.lower(chave)=='colormap'):
            colormap=kargs[chave]
   
    # https://matplotlib.org/stable/users/explain/colors/colormaps.html
    colors = plt.get_cmap(colormap)(np.linspace(0.2, 0.7, len(etiquetas)))
    
    # plotar...
    fig, ax = plt.subplots(figsize=(12,10))
    plt.title(titulo, fontdict={'weight':'bold'})
    ax.pie(valores, 
           colors=colors,
           radius=3, 
           center=(4, 4),
           textprops={'size':12, 'weight':'bold'},
           wedgeprops={"linewidth": 3, "edgecolor": "white"}, 
           frame=True,
           labels=etiquetas,
           autopct='%1.1f%%',
          )

    #ax.legend(labels=etiquetas,
    #          title=titulo,
    #          loc='upper right',
    #          bbox_to_anchor=(.5, 1, 1, 0)
    #         )

    ax.set(xlim=(0, 8), xticks=np.arange(1, 8),
           ylim=(0, 8), yticks=np.arange(1, 8))
    plt.savefig(nomeArquivo, dpi=300)
    plt.show()

In [ ]:
#
pieGraph(dataPieGraph(ins['genero'])
        ,titulo='Gênero'
        ,nomeArquivo='./Imagens/Genero.jpg'
        ,colorMap='autumn'
        )

pieGraph(dataPieGraph(ins['escolaridade2'])
        ,titulo='Escolaridade'
        ,nomeArquivo='./Imagens/Escolaridade.jpg'
        ,colorMap='coolwarm'
        )

pieGraph(dataPieGraph(ins['cidade'])
        ,titulo='Cidade'
        ,nomeArquivo='./Imagens/Cidade.jpg'
        ,colorMap='Set1'
        )

pieGraph(dataPieGraph(ins['observacao'])
        ,titulo='Observação'
        ,nomeArquivo='./Imagens/Observacao.jpg'
        ,colorMap='cividis'
        )

In [ ]:
dataPie=dataComputeByRange([{'min':0,'max':19},{'min':20,'max':29},{'min':30,'max':39},
                         {'min':40,'max':49},{'min':50,'max':59},{'acima':60}],
                        ins['idade'] 
                   )
pieGraph(dataPie
        ,titulo='Por idade'
        ,nomeArquivo='./Imagens/PorIdade.jpg'
        ,colorMap='BuPu')

In [ ]:
# https://stackoverflow.com/questions/50807744/apply-css-class-to-pandas-dataframe-using-to-html
def geraHTML(dataFrame, arquivoHTML = 'index.html', 
             cssStyle = 'style.css', titulo = 'Html'):
    html_string = '''
        <!DOCTYPE html>
        <html lang='pt-br'>
          <head>
              <title>{titulo}</title>
          </head>
          <link rel="stylesheet" type="text/css" href="{css}"/>
          <body>
                {tabela}
          </body>
        </html>
    '''
    with open(arquivoHTML, 'w') as arquivo:
        arquivo.write(html_string.format(titulo=titulo,
                               tabela=dataFrame.to_html(
                                       index=False,
                                       justify='center',
                                       classes='mystyle'),
                               css=cssStyle
                              ))
    
insHtml=ins[['nroInscricao','nome','cpfAjustado','dataNascAjustada','idade','genero',
             'escolaridade2','instituicao','telefoneAjustado','emailValidado',
             'cidade','cepAjustado','observacao']]       
insHtml.columns=[['Nro.Inscricao','Nome','CPF','Data Nascimento','Idade','Gênero',
                  'Escolaridade','Instituição','Telefone/Cel','E-Mail',
                  'Cidade','CEP','Observação']]  
        
geraHTML(insHtml, 
         './html/Hackathon 2024 - Inscrições.html','../css/df_style.css',
         'Hackathon 2024 - Inscrições')

In [ ]:
#### End